In [2]:
import os
import pandas as pd
from flask import Flask, jsonify, request, render_template
import json
import dill as pickle

app = Flask(__name__)


def load_pk():
    model = pickle.load(open("xgb_model.pkl", 'rb'))
    columns = pickle.load(open("columns.pkl", 'rb'))
    return (model, columns)


def json2df(json_data):
    print("   JSON to DataFrame Process")
    df = pd.DataFrame()
    try:
        # If the json data consist of more than 2 records
        for col in json_data:
            for row in json_data[col]:
                df.loc[int(row), col] = json_data[col][row]
    except:
        # If the json_data is 1 record only
        for col in json_data:
            df.loc[1, col] = json_data[col]
    finally:
        print(df)
        return df


@app.route("/")
def start():
    return render_template("home.html")  


@app.route('/predict', methods=['POST'])
def apicall():
    try:
        json_ = request.json
        test = json2df(json_)

    except Exception as e:
        raise e
    if test.empty:
        return jsonify({})
    else:
        print("   Predict on Process")
        predictions = model.predict(test)
        # predictions = pd.DataFrame(predictions, columns=["Prediction"])
        # responses = jsonify(predictions.to_json(orient='records'))
        responses = jsonify({"prediction": str(predictions)})
        responses.status_code = 200
        return responses


if __name__ == "__main__":
    print("   Loading Machine Learning Model...")
    (model, columns) = load_pk()
    app.run(host="0.0.0.0",port=5000,debug=True)

   Loading Machine Learning Model...
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

c:\users\hendra\opencv\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
